In [1]:
import tkinter as tk
from tkinter import messagebox
import datetime

# ------------------------- Core Functions -------------------------

def validate_inputs(balance, monthly_payment, interest_rate, end_date):
    try:
        if balance <= 0:
            return "Balance must be a positive number."
        if monthly_payment is not None and monthly_payment <= 0:
            return "Monthly Payment must be greater than zero."
        if interest_rate is not None and interest_rate <= 0:
            return "Interest Rate must be greater than zero."
        if end_date and not isinstance(end_date, datetime.date):
            return "Invalid End Date format."
        if end_date and end_date <= datetime.date.today():
            return "End Date must be in the future."
    except Exception as e:
        return f"Validation Error: {e}"
    return None

def calculate_monthly_payment(balance, interest_rate, end_date):
    today = datetime.date.today()
    months = (end_date.year - today.year) * 12 + (end_date.month - today.month)
    if months <= 0:
        return None
    monthly_rate = (interest_rate / 100) / 12
    try:
        payment = (balance * monthly_rate) / (1 - (1 + monthly_rate) ** -months)
        return round(payment, 2)
    except:
        return None

def calculate_end_date(balance, monthly_payment, interest_rate):
    if monthly_payment <= 0:
        return None
    monthly_rate = (interest_rate / 100) / 12
    today = datetime.date.today()
    current = today
    count = 0
    while balance > 0 and count < 1200:
        interest = balance * monthly_rate
        balance = balance + interest - monthly_payment
        balance = max(0, round(balance, 2))
        current = increment_month(current)
        count += 1
    return current if count < 1200 else None

def calculate_interest_rate(balance, monthly_payment, end_date):
    today = datetime.date.today()
    months = (end_date.year - today.year) * 12 + (end_date.month - today.month)
    if months <= 0 or monthly_payment <= 0:
        return None

    # Estimate whether the loan is repayable at all
    if monthly_payment <= balance / months:
        return None

    low = 0.0001
    high = 100.0
    for _ in range(300):  # higher precision
        mid = (low + high) / 2
        estimated = calculate_monthly_payment(balance, mid, end_date)
        if estimated is None:
            return None
        if abs(estimated - monthly_payment) < 0.01:
            return round(mid, 4)
        if estimated > monthly_payment:
            high = mid
        else:
            low = mid
    return None

def calculate_interest_breakdown(balance, monthly_payment, interest_rate, end_date):
    today = datetime.date.today()
    monthly_rate = (interest_rate / 100) / 12
    result = []
    while balance > 0 and today <= end_date:
        interest = balance * monthly_rate
        balance += interest
        balance -= monthly_payment
        balance = max(0, round(balance, 2))
        result.append(f"{today} | Balance: ${balance:.2f}")
        today = increment_month(today)
    return "\n".join(result)

def increment_month(date):
    year = date.year + (date.month // 12)
    month = (date.month % 12) + 1
    return datetime.date(year, month, 1)

# ------------------------- GUI Functions -------------------------

def reset_fields():
    entry_balance.config(state=tk.NORMAL)
    entry_interest_rate.config(state=tk.NORMAL)
    entry_end_date.config(state=tk.NORMAL)
    entry_monthly_payment.config(state=tk.NORMAL)

    entry_balance.delete(0, tk.END)
    entry_interest_rate.delete(0, tk.END)
    entry_monthly_payment.delete(0, tk.END)
    entry_end_date.delete(0, tk.END)
    entry_end_date.insert(0, (datetime.date.today() + datetime.timedelta(days=365)).strftime('%Y-%m-%d'))

    text_result.config(state=tk.NORMAL)
    text_result.delete("1.0", tk.END)
    text_result.config(state=tk.DISABLED)

def set_mode(mode):
    global current_mode
    current_mode = mode
    reset_fields()
    label_title.config(text=f"Calculating: {mode}")

    entry_balance.config(state=tk.NORMAL)
    entry_interest_rate.config(state=tk.NORMAL)
    entry_end_date.config(state=tk.NORMAL)
    entry_monthly_payment.config(state=tk.NORMAL)

    if mode == "Monthly Payment":
        entry_monthly_payment.config(state=tk.DISABLED)
    elif mode == "Interest Rate":
        entry_interest_rate.config(state=tk.DISABLED)
    elif mode == "End Date":
        entry_end_date.config(state=tk.DISABLED)

    button_submit.config(state=tk.NORMAL)

def run_calculation():
    try:
        balance = float(entry_balance.get())
        interest_rate = float(entry_interest_rate.get()) if entry_interest_rate['state'] == tk.NORMAL else None
        monthly_payment = float(entry_monthly_payment.get()) if entry_monthly_payment['state'] == tk.NORMAL else None
        end_date = datetime.datetime.strptime(entry_end_date.get(), "%Y-%m-%d").date() if entry_end_date['state'] == tk.NORMAL else None

        if current_mode == "Monthly Payment":
            error = validate_inputs(balance, 1, interest_rate, end_date)
            if error: return messagebox.showerror("Error", error)
            result = f"Monthly Payment: ${calculate_monthly_payment(balance, interest_rate, end_date):.2f}"

        elif current_mode == "End Date":
            error = validate_inputs(balance, monthly_payment, interest_rate, None)
            if error: return messagebox.showerror("Error", error)
            end = calculate_end_date(balance, monthly_payment, interest_rate)
            result = f"End Date: {end.strftime('%Y-%m-%d')}" if end else "Unable to determine end date."

        elif current_mode == "Interest Rate":
            error = validate_inputs(balance, monthly_payment, 1, end_date)
            if error: return messagebox.showerror("Error", error)
            rate = calculate_interest_rate(balance, monthly_payment, end_date)
            if rate:
                result = f"Estimated Interest Rate: {rate:.4f}%"
            else:
                result = "❌ Could not determine interest rate. Try increasing monthly payment or extending end date."

        elif current_mode == "Interest Breakdown":
            error = validate_inputs(balance, monthly_payment, interest_rate, end_date)
            if error: return messagebox.showerror("Error", error)
            result = calculate_interest_breakdown(balance, monthly_payment, interest_rate, end_date)

        else:
            result = "Unknown mode selected."

        text_result.config(state=tk.NORMAL)
        text_result.delete("1.0", tk.END)
        text_result.insert(tk.END, result)
        text_result.config(state=tk.DISABLED)

    except Exception as e:
        messagebox.showerror("Error", f"Something went wrong: {e}")

# ------------------------- GUI Setup -------------------------

root = tk.Tk()
root.title("Smart Loan Calculator")
root.geometry("720x720")

label_title = tk.Label(root, text="Choose a calculation mode", font=("Arial", 16))
label_title.grid(row=0, column=0, columnspan=2, pady=10)

tk.Button(root, text="Calculate Monthly Payment", width=30, command=lambda: set_mode("Monthly Payment")).grid(row=1, column=0, pady=5)
tk.Button(root, text="Calculate End Date", width=30, command=lambda: set_mode("End Date")).grid(row=1, column=1, pady=5)
tk.Button(root, text="Calculate Interest Rate", width=30, command=lambda: set_mode("Interest Rate")).grid(row=2, column=0, pady=5)
tk.Button(root, text="Interest Breakdown", width=30, command=lambda: set_mode("Interest Breakdown")).grid(row=2, column=1, pady=5)

tk.Label(root, text="Loan Balance ($):").grid(row=3, column=0, sticky='e', padx=10, pady=5)
entry_balance = tk.Entry(root)
entry_balance.grid(row=3, column=1, padx=10)

tk.Label(root, text="Interest Rate (%):").grid(row=4, column=0, sticky='e', padx=10, pady=5)
entry_interest_rate = tk.Entry(root)
entry_interest_rate.grid(row=4, column=1, padx=10)

tk.Label(root, text="End Date (YYYY-MM-DD):").grid(row=5, column=0, sticky='e', padx=10, pady=5)
entry_end_date = tk.Entry(root)
entry_end_date.insert(0, (datetime.date.today() + datetime.timedelta(days=365)).strftime('%Y-%m-%d'))
entry_end_date.grid(row=5, column=1, padx=10)

tk.Label(root, text="Monthly Payment ($):").grid(row=6, column=0, sticky='e', padx=10, pady=5)
entry_monthly_payment = tk.Entry(root)
entry_monthly_payment.grid(row=6, column=1, padx=10)

button_submit = tk.Button(root, text="Calculate", command=run_calculation, state=tk.DISABLED)
button_submit.grid(row=7, column=0, columnspan=2, pady=10)

text_result = tk.Text(root, height=15, width=85, wrap=tk.WORD, state=tk.DISABLED)
text_result.grid(row=8, column=0, columnspan=2, padx=10, pady=10)

current_mode = ""
root.mainloop()
